<a href="https://colab.research.google.com/github/AnnaGass/DSA-SleepHealthData/blob/main/01_DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# @title # Select the subject to be used { display-mode: "both" }
subject = '020'

# Install python libraries and load packages

In [21]:
# @title Install needed libraries { display-mode: "both" }
!pip install pyedflib
!pip install pandas

!pip install pandas scikit-learn

In [22]:
# @title Import libraries { display-mode: "both" }
import subprocess
import pyedflib
import csv
import os
import shutil

import pandas as pd

from datetime import timedelta
from datetime import datetime

In [23]:
# @title Define constants { display-mode: "both" }
base_url = "https://physionet.org/files/ucddb/1.0.0/ucddb" + subject
orig_data = "/content/physionet.org/files/ucddb/1.0.0/ucddb" + subject

output_directory = "results/subject" + subject + "/"
data_prep_dir = output_directory + "data_preparation/"

base_data = output_directory + "base_data" + subject + ".csv"

In [24]:
# @title Create output directory for subject { display-mode: "both" }

# Create folders if not exists
if not os.path.exists(output_directory):
  os.makedirs(output_directory)

if not os.path.exists(data_prep_dir):
  os.makedirs(data_prep_dir)

# Import data

In [25]:
# @title Import the data from physionet { display-mode: "both" }

base_command = "wget -r -N -c -np "

rec_path = base_url + ".rec"
sleep_annotation_path = base_url + "_stage.txt"
apnea_annotation_path = base_url + "_respevt.txt"

command = base_command + rec_path
result = subprocess.run(command, shell=True, capture_output=True, text=True)

command = base_command + sleep_annotation_path
result = subprocess.run(command, shell=True, capture_output=True, text=True)

command = base_command + apnea_annotation_path
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# Prepare the data

In [26]:
# @title Extract signals from .rec file

# open the input file
rec_file_path = orig_data + ".rec"
f = pyedflib.EdfReader(rec_file_path)

# Specify the output file name
signals_csv = data_prep_dir + "01_orig_signals.csv"

# Specify which signals to use
spo2_index = 6      # taken from the headers
pulse_index = 13    # taken from the headers

# Initialize an empty list to store your values
data = []

# file properties
start_time = f.getStartdatetime()
file_duration = f.getFileDuration()
signal_labels = f.getSignalLabels()

# SpO2 information
spo2_header = f.getSignalHeader(spo2_index)
spo2_signals = f.readSignal(spo2_index)

# Pulse information
pulse_header = f.getSignalHeader(pulse_index)
pulse_signals = f.readSignal(pulse_index)

# Calculate frequency
time_between_meassurements = file_duration / len(pulse_signals)

# Iterate and add the increment to the datetime on each iteration
for i in range(len(pulse_signals)):
    date_str = start_time.strftime('%Y-%m-%d')
    if start_time.microsecond != 0:
       time_str = start_time.strftime('%H:%M:%S.%f')
    else:
       time_str = start_time.strftime('%H:%M:%S')
    data.append([date_str, time_str, spo2_signals[i], pulse_signals[i]])
    start_time += timedelta(seconds=time_between_meassurements)

# Write the data to a CSV file
with open(signals_csv, 'w', newline='') as file:
    # Create a CSV writer object for the output file
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Date', 'Time', 'SpO2', 'Pulse'])
    writer.writerows(data)
f.close()

In [27]:
# @title Extract data from respevt.txt file

# Specify the input file name
annotations_path = orig_data + "_respevt.txt"

# Specify the output file name
output_csv_annotations = data_prep_dir + "02_apnea_annotations.csv"

# Open the input file
with open(annotations_path, 'r') as infile:
    # Skip the first two lines (header)
    next(infile)
    next(infile)
    next(infile)

    # Open a CSV file for writing with more parameters
    with open(output_csv_annotations, 'w', newline='') as outfile_extra:
        # Create a CSV writer object for the output file
        writer = csv.writer(outfile_extra)

        # Write the header
        writer.writerow(['Time', 'Apnea'])

        # Process each line in the input file
        for line in infile:
            # Split the line into columns
            columns = line.split()

            # Check if the line has enough columns
            if len(columns) >= 5:

                # Extract the required columns
                time_str = columns[0]
                event_type = columns[1]
                duration_str = columns[2] if columns[2] != "EVENT" else columns[4]


                # Parse duration into seconds
                duration_seconds = int(duration_str)

                # Parse time string into datetime object
                time = datetime.strptime(time_str, '%H:%M:%S')

                # Iterate over the duration and add one second to the time in each iteration
                for _ in range(duration_seconds):

                    # Write the time, event type, and duration to the CSV file
                    writer.writerow([time.strftime('%H:%M:%S'), event_type])

                    # Add one second to the time
                    time += timedelta(seconds=1)

In [28]:
# @title Extract data from stage.txt file

# Specify the input file name
input_stage = orig_data + "_stage.txt"

# Specify the output file name
output_csv_stage = data_prep_dir + "03_sleep_stages.csv"

# Open the input text file in read mode and the output CSV file in write mode
with open(input_stage, 'r') as infile:
  with open(output_csv_stage, 'w', newline='') as outfile_stage:
        # Create a CSV writer object for the output file
        writer = csv.writer(outfile_stage)

        # Write the header
        writer.writerow(['Sleep Stage'])

        # Process each line in the input file
        for line in infile:
            # Remove leading and trailing whitespace and split the line into columns
            line = line.strip()
            # Write the sleep stage value to the CSV file
            writer.writerow([line])

In [29]:
# @title Sort the data in the orig_signals.csv and merge it with sleep_stages.csv and apnoe_annotations.csv

merged_csv = data_prep_dir + "04_merged.csv"

# Load data from the _signals.csv file
df_signals = pd.read_csv(signals_csv)

# Sort the data to keep only the rows where the time values have seconds only
df_signals = df_signals[df_signals['Time'].str.contains(r'^\d{2}:\d{2}:\d{2}$')]
df_signals_withoutmiliseconds = df_signals.copy()

# Load data from the apnea_annotations.csv file and merge it with the filtered signal data using the 'Time' column
df_apnea_anno = pd.read_csv(output_csv_annotations)
df_merged_anno = pd.merge(df_signals_withoutmiliseconds, df_apnea_anno, on='Time', how='left')

# =================================================================================================

# Load data from the sleep_stages.csv file
df_apnea_stage = pd.read_csv(output_csv_stage, dtype=int)

# Initialize an empty list to store combined data
combined_data = []

for i in range(len(df_signals_withoutmiliseconds)):
    # Calculate the index for the data from df_apnea_stage based on the 30-second intervals
    apnea_index = i // 30

    # Ensure the index does not exceed the number of rows in df_apnea_stage
    apnea_index = min(apnea_index, len(df_apnea_stage) - 1)

    # Append the corresponding entry from df_apnea_stage to the combined data list
    combined_data.append(df_apnea_stage.iloc[apnea_index].tolist())

# Convert the combined data list into a DataFrame
df_combined_stage = pd.DataFrame(combined_data, columns=df_apnea_stage.columns)

# Merge the combined data with the merged signal and annotation data
df_merged_stage = pd.concat([df_merged_anno, df_combined_stage], axis=1)

# Save the merged data to a new CSV file
df_merged_stage.to_csv(merged_csv, index=False)

In [30]:
# @title Remove all datasets where SpO2 < 70 or 40 < Pulse < 180 (Meassurement error)

# Load the CSV file into a DataFrame
df = pd.read_csv(merged_csv)

merged_clean_csv = data_prep_dir + "05_merged_clean.csv"

spo2_threshold = 70
pulse_lower_threshold = 40
pulse_upper_threshold = 180

df = df[df['SpO2'] >= spo2_threshold]
df = df[df['Pulse'] >= pulse_lower_threshold]
df = df[df['Pulse'] <= pulse_upper_threshold]

# Save the filtered DataFrame to a new CSV file
df.to_csv(merged_clean_csv, index=False)

In [31]:
# @title Categorize SpO2
# @markdown 0-70 = Invalid, 70-80 = Severe Hypoxia, 80-90 = Critical, 90-94 = Decreased, 94-100 = Normal

# Load the CSV file into a DataFrame
df = pd.read_csv(merged_clean_csv)

# Define the name of the output file
spo2_categorized_csv = data_prep_dir + "06_spo2_categorized.csv"

# Define the categorization function
def categorize_spo2(value):
    if value >= 0 and value < 70:
        return 'Invalid'
    elif value >= 70 and value < 80:
        return 'Severe Hypoxia'
    elif value >= 80 and value < 90:
        return 'Critical'
    elif value >= 90 and value < 94:
        return 'Decreased'
    elif value >= 94 and value <= 100:
        return 'Normal'
    else:
        return 'Out of Range'  # In case the value is out of the expected range

# Apply the categorization function to the relevant column
df['SpO2 Category'] = df['SpO2'].apply(categorize_spo2)

# Save the DataFrame to a new CSV file (optional)
df.to_csv(spo2_categorized_csv, index=False)

In [32]:
# @title Categorize Pulse
# @markdown Bin size = (max_value - min_value) / number_of_bins
number_of_bins = 7 # @param {type:"integer"}

# Load the CSV file into a DataFrame
df = pd.read_csv(spo2_categorized_csv)

# Define the name of the output file
pulse_categorized_csv = data_prep_dir + "07_pulse_categorized.csv"

# Find out the smallest and biggest values for pulse
min_value = df['Pulse'].min()
max_value = df['Pulse'].max()

# Calculate the bin size
bin_size = (max_value - min_value) / number_of_bins

# Create a list of bin edges
bin_edges = [min_value + i * bin_size for i in range(number_of_bins + 1)]

# Function to assign value to correct bin
def categorize_pulse(value):
    for i in range(number_of_bins):
        if bin_edges[i] <= value < bin_edges[i + 1]:
            return f'[{bin_edges[i]:.2f} - {bin_edges[i + 1]:.2f}]'
    return f'[{bin_edges[-2]:.2f} - {bin_edges[-1]:.2f}]'  # For the last bin

# Apply the categorization function to the relevant column
df['Pulse Range'] = df['Pulse'].apply(categorize_pulse)

# Save the filtered and categorized DataFrame to a new CSV file
df.to_csv(pulse_categorized_csv, index=False)


In [33]:
# @title Set the correct text for sleep phase and remove missing values

# Load the CSV file into a DataFrame
df = pd.read_csv(pulse_categorized_csv)

# Define the name of the output file
sleep_stage_text_csv = data_prep_dir + "08_sleep_stage_text.csv"

# Define the mapping for the "Sleep Stage" column
sleep_stage_mapping = {
    0: 'Wake',
    1: 'REM',
    2: 'Stage 1',
    3: 'Stage 2',
    4: 'Stage 3',
    5: 'Stage 4',
    6: 'Artifact',
    7: 'Indeterminate',
    8: 'Invalid'
}

# Apply the mapping to the "Sleep Stage" column
df['Sleep Stage Text'] = df['Sleep Stage'].map(sleep_stage_mapping)

# Save the updated DataFrame to a new CSV file
df.to_csv(sleep_stage_text_csv, index=False)

In [34]:
# @title Simplify target class

# Load the CSV file into a DataFrame
df = pd.read_csv(sleep_stage_text_csv)

# Define the name of the output file
final_csv = data_prep_dir + "09_final.csv"

def normalize_apnea_column(value):
    if str(value).startswith('AP'):
        return 'Apnea'
    elif str(value).startswith('HYP'):
        return 'Hypoapnea'
    else:
        return 'Nothing'

# Apply the normalization function to the target class
df['Apnea'] = df['Apnea'].apply(normalize_apnea_column)

# Save the filtered, categorized, and normalized DataFrame to a new CSV file
df.to_csv(final_csv, index=False)

In [35]:
# @title Copy final csv to results folder

# Use shutil.copy to copy the file
shutil.copy(final_csv, base_data)

'results/subject020/base_data020.csv'